<div align="right"><i>Peter Norvig<br>2012; updated 18 August 2019</i></div>

# Weighing Twelve Balls on a Balance Scale

> *You are given twelve identical-looking balls and a two-sided scale. One of the balls is of a different weight, although you don't know whether it's lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it's lighter or heavier?*

This is a traditional brain-teaser puzzle, meant to be solved with paper and pencil. 
But I want to solve not just this specific puzzle, but related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the odd ball might be heavier, lighter, or either. For that I'll ned a program. (I originally solved wrote this program in 2012, but am republishing it here in revised form because the problem was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019.)

# Design

Here are the concepts I'm dealing with:

- **balls**: In the general case I have N balls. I'll represent with a list like `[1, 2, 3]` for N = 3.
- **oddballs**: Exactly one of the balls is **odd** in its weight; if ball N is heavier, I'll represent that as +N; if it is lighter, as -N. With N = 3, I will represent the list of possible oddballs as `[+1, -1, +2, -2, +3, -3]`.
- **puzzle**: A specific puzzle declares the number of balls, the maximum number of weighings allowed, and the possible oddballs. 
- **weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result will be that the left side is greater than, equal to, or less than the right in weight. I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **weight**: I'll arbitrarily say that a normal ball weighs 100, a lighter ball 99, and a heavier ball 101.
- **solution**: Given a puzzle, a solution is a data structure that can correctly  states the number of balls, the number of weighings allowed, and whether the odd ball can be lighter, heavier, or either. For example, the original puzzle is solved with a call to `solve(12, 3, {+1, -1})`. The solution is a **strategy tree**.
- **strategy tree**: a tree where each node is either a leaf node consisting of an *oddball* integer, or is an interior node with 5 components: the balls to be placed on the left and right side of the scale, and a subtree for each of the three possible outcomes: `'gt'`, `'eq'`, or `'lt'`. The constructor `Tree(L, R, gt, eq, lt)` creates a tree, where `L` and `R` are collections of balls, and `gt`, `eq`, and `lt` are trees.
- **following a path in a tree**: I'll use `follow(tree, oddball)` to say "follow the path through the tree, at each weighing assuming the given oddball, and return the leaf node reached&mdash;the oddball that the tree predicts.
- **valid tree**: a tree is a valid solution if no branch uses more than the allowable number of weighings, and if, for every possible oddball, following the path through the tree gives the correct oddball as the answer. We'll delay the discussion of how to find a valid tree until later.


# Implementation

Let's start implementing the concepts:

In [1]:
from collections import namedtuple
import random

#### Types

Ball = Oddball = int

class Puzzle:
    "Represent a specific ball-weighing puzzle."
    def __init__(self, N=12, weighings=3, oddities={+1, -1}):
        self.N = N
        self.weighings = weighings
        self.balls = list(range(1, N + 1))
        self.oddballs = [b * o for b in self.balls for o in oddities]  
        
Tree = namedtuple('Tree', 'L, R, gt, eq, lt')

#### Functions
    
def weigh(L, R, oddball) -> str:
    "Weigh balls L against balls R, given the oddball; return 'gt', 'eq', or 'lt'."
    diff = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return ('gt' if diff > 0 else
            'lt' if diff < 0 else
            'eq')

def weight(ball, oddball) -> int: 
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Tree:
    "Return a valid tree; one that solves the puzzle."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    assert valid(tree, puzzle)
    return tree
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if isinstance(tree, Oddball):
        return tree
    else:
        result = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, result), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth of a strategy tree."
    return (0 if isinstance(tree, Oddball) else 
            1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

Let's try out some of these functions:

In [2]:
p12 = Puzzle(12) # The original puzzle with 12 balls

print(p12.weighings)
print(p12.balls)
print(p12.oddballs)

3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12]


In [3]:
# If we weigh balls 1, 2 against 3, 4, and the oddball is that 5 is lighter, the result should be 'eq'
weigh([1, 2], [3, 4], -5)

'eq'

In [4]:
weight(1, oddball=-5)

100

In [5]:
weight(5, oddball=-5)

99

# Strategy for Finding a Valid Tree

Now for the tricky part. We want to find a valid tree to solve a puzzle. The key idea is that a **weighing** gives us information by making a **partition** of the possible **oddballs** into entries for eaach of the three possible weighing results: `gt`, `eq`, or `lt`. Subsequent subtrees can handle each of the partitions.


In [6]:
def partition(L, R, oddballs) -> dict:
    "Give all the possible outcomes of weighing L versus R."
    part = dict(gt=[], eq=[], lt=[])
    for odd in oddballs:
        part[weigh(L, R, odd)].append(odd)
    return part

For example, with 12 balls, if we weigh balls 1 and 2 on the left versus 11 and 12 on the right, then there are 4 ways  the left side can be greater than the right: either 1 or 2 is heavier or 11 or 12 is lighter. Similarly there are 4 ways of getting a `lt` weighing result. The remaining 16 possible oddballs&mdash;balls 3 through 10 being either heavier or lighter&mdash;show up in the `eq` entry of the partition:

In [7]:
partition([1, 2], [11, 12], p12.oddballs)

{'gt': [1, 2, -11, -12],
 'eq': [3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10],
 'lt': [-1, -2, 11, 12]}

If this was the first weighing in our strategy tree, could we go on to solve the puzzle in 3 weighings? **No!** Any one weighing can at best partition the remaining possibilities into 3 equal entries. To solve the puzzle, we need every path in the tree to end up with only one possibility. So any two weighings can handle at most  3 &times; 3 = 9 possibilities; here we have 16, which is too many. We call this a **bad partition**.

The following is a **good partition** because each of the entries has 8 possibilities, and 8 is less than 9. (Note: being a good partition does not guarantee that the problem is solvable from there; but being a bad partition guarantees that it is not solvable.)

In [8]:
partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)

{'gt': [1, 2, 3, 4, -9, -10, -11, -12],
 'eq': [5, -5, 6, -6, 7, -7, 8, -8],
 'lt': [-1, -2, -3, -4, 9, 10, 11, 12]}

So now we have a viable approach to implementing `find_tree`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we will reduce the number of oddball possibilities according to the partition, and the number of remaining weighings by 1 each time.
   - At each step we will randomly select two groups of balls, `L` and `R`, to be weighed.
   - We will then see what partition `L` and `R` gives us, and whether the partition is good or bad.
   - If the partition is bad, try another random selection of `L` and `R`.
   - Use a **greedy** approach where we accept the first good partition.
      (If we don't find a good partition after 10,000 tries, we give up.)
   - Once we have a good partition, we recursively find a tree for each of the branches of the partition.

In [9]:
def find_tree(puzzle, oddballs, weighings) -> Tree or Oddball:
    "Find a strategy tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return oddballs[0] # One oddball possibility left; we're done
    elif len(oddballs) == 0 or weighings == 0:
        return 0           # No valid strategy or an impossible situation
    else:
        L, R, part = find_good_partition(puzzle, oddballs, weighings - 1)
        subtrees = {r: find_tree(puzzle, part[r], weighings - 1) for r in part}
        return Tree(L, R, **subtrees)
    
def find_good_partition(puzzle, oddballs, weighings) -> tuple:
    "Return (L, R, partition) such that no partition entry has more than 3**weighings oddballs."
    for _ in range(10000): 
        L, R = random_LR(puzzle, oddballs)
        part = partition(L, R, oddballs)
        if all(len(entry) <= 3 ** weighings for entry in part.values()):
            return L, R, part
    raise ValueError('find_good_partition: not found')
    
def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then randomly pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    random.shuffle(puzzle.balls) 
    return sorted(puzzle.balls[:B]), sorted(puzzle.balls[-B:])

Here we see that `find_good_partition` does its job:

In [10]:
find_good_partition(p12, p12.oddballs, 2)

([2, 3, 7, 9],
 [1, 6, 10, 11],
 {'gt': [-1, 2, 3, -6, 7, 9, -10, -11],
  'eq': [4, -4, 5, -5, 8, -8, 12, -12],
  'lt': [1, -2, -3, 6, -7, -9, 10, 11]})

But it uses `random`, so it won't get the same result every time:

In [11]:
find_good_partition(p12, p12.oddballs, 2)

([1, 2, 8, 11],
 [3, 4, 7, 10],
 {'gt': [1, 2, -3, -4, -7, 8, -10, 11],
  'eq': [5, -5, 6, -6, 9, -9, 12, -12],
  'lt': [-1, -2, 3, 4, 7, -8, 10, -11]})

# Solving Some Puzzles

Now we're ready to solve puzzles!

In [12]:
solve(p12)

Tree(L=[4, 7, 10, 12], R=[3, 6, 8, 9], gt=Tree(L=[1, 4, 5, 7], R=[2, 6, 10, 12], gt=Tree(L=[7, 10], R=[3, 4], gt=7, eq=-6, lt=4), eq=Tree(L=[3, 5, 6, 10], R=[1, 4, 7, 9], gt=-9, eq=-8, lt=-3), lt=Tree(L=[3, 5, 6, 8], R=[2, 4, 10, 11], gt=0, eq=12, lt=10)), eq=Tree(L=[1, 3, 11, 12], R=[5, 6, 7, 9], gt=Tree(L=[2, 8, 12], R=[1, 4, 5], gt=-5, eq=11, lt=1), eq=Tree(L=[2, 4, 11, 12], R=[1, 3, 5, 10], gt=2, eq=0, lt=-2), lt=Tree(L=[1, 2, 6], R=[3, 9, 11], gt=-11, eq=5, lt=-1)), lt=Tree(L=[5, 7, 9, 10], R=[1, 2, 4, 12], gt=Tree(L=[3, 7, 8], R=[2, 9, 12], gt=-12, eq=-4, lt=9), eq=Tree(L=[1, 4, 8], R=[2, 6, 10], gt=8, eq=3, lt=6), lt=Tree(L=[12], R=[7], gt=-7, eq=-10, lt=0)))

OK, that's hard to read&mdash;my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the odd ball can only be lighter:

In [13]:
tree = solve(Puzzle(3, 1, {-1}))
tree

Tree(L=[1], R=[3], gt=-3, eq=-2, lt=-1)

This tree says you weigh one ball against another (leaving the third unweighed), and the three possible weighing results tell you which of the three balls  is lighter.

To handle bigger puzzles, let's make the trees easier to read by producing formatted, indented output. Also, note that at the top node of a tree, there's no sense randomly shuffling the balls; the only choice that matters is how many balls, `B`, to put on each side. I'll alter `random_LR` to do that; that should also make trees easier to understand.

In [14]:
def do(puzzle):
    "Print the solution to the puzzle as indented text."
    print(indented(solve(puzzle)))
    
def indented(tree, i=0, prefix=''):
    "Pretty, indented string of a strategy tree."
    if isinstance(tree, Tree):
        subtrees = f"{indented(tree.gt, i+1, '>:')} {indented(tree.eq, i+1, '=:')} {indented(tree.lt, i+1, '<:')}"
        indent   = '' if i == 0 else ('\n' + " " * 5 * i)
        return f'{indent}{prefix}({items(tree.L)} ⟘ {items(tree.R)}) ➔ {subtrees})'
    elif tree == 0:
        return f'{prefix}0'
    else:
        return f'{prefix}{tree:+d}'
    
def items(collection): return '·'.join(map(str, collection))

def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    if oddballs == puzzle.oddballs:
        puzzle.balls.sort()
    else:
        random.shuffle(puzzle.balls) 
    return sorted(puzzle.balls[:B]), sorted(puzzle.balls[-B:])

In [15]:
# 3 balls, 1 weighing, only lighter
do(Puzzle(3, 1, {-1}))

(1 ⟘ 3) ➔ >:-3 =:-2 <:-1)


In [16]:
# 3 balls, 2 weighings, lighter or heavier
do(Puzzle(3, 2))

(1 ⟘ 3) ➔ 
     >:(2 ⟘ 1) ➔ >:0 =:-3 <:+1) 
     =:(1 ⟘ 2) ➔ >:-2 =:0 <:+2) 
     <:(2 ⟘ 3) ➔ >:0 =:-1 <:+3))


In [17]:
# 12 balls
do(p12)

(1·2·3·4 ⟘ 9·10·11·12) ➔ 
     >:(3·6·11·12 ⟘ 2·8·9·10) ➔ 
          >:(2·3·8·9 ⟘ 1·5·6·7) ➔ >:+3 =:-10 <:-9) 
          =:(2·4·8·9 ⟘ 6·10·11·12) ➔ >:+4 =:+1 <:0) 
          <:(1·6·11 ⟘ 3·7·12) ➔ >:-12 =:+2 <:-11)) 
     =:(1·5·7·10 ⟘ 2·4·8·11) ➔ 
          >:(1·7·11 ⟘ 5·9·10) ➔ >:+7 =:-8 <:+5) 
          =:(1·6·7·12 ⟘ 3·4·9·11) ➔ >:+6 =:0 <:-6) 
          <:(3·6·9 ⟘ 2·7·8) ➔ >:-7 =:-5 <:+8)) 
     <:(3·7·10·11 ⟘ 4·8·9·12) ➔ 
          >:(2·7·8 ⟘ 4·6·10) ➔ >:-4 =:+11 <:+10) 
          =:(3·11·12 ⟘ 1·6·8) ➔ >:-1 =:-2 <:0) 
          <:(1·3·12 ⟘ 4·5·11) ➔ >:+12 =:+9 <:-3)))


In [18]:
# A different random solution
do(p12)

(1·2·3·4 ⟘ 9·10·11·12) ➔ 
     >:(2·4·7·12 ⟘ 1·3·6·9) ➔ 
          >:(4·5·9·10 ⟘ 1·3·6·8) ➔ >:+4 =:+2 <:-9) 
          =:(4·5·6·11 ⟘ 1·3·9·10) ➔ >:-10 =:0 <:-11) 
          <:(1·5·9·11 ⟘ 3·4·8·10) ➔ >:+1 =:-12 <:+3)) 
     =:(1·5·6·7 ⟘ 2·4·11·12) ➔ 
          >:(2·4·5·11 ⟘ 1·7·8·9) ➔ >:+5 =:+6 <:+7) 
          =:(1·4·8·12 ⟘ 5·7·10·11) ➔ >:+8 =:0 <:-8) 
          <:(5·10 ⟘ 3·7) ➔ >:-7 =:-6 <:-5)) 
     <:(2·7·8·9 ⟘ 1·3·6·10) ➔ 
          >:(6·10 ⟘ 3·9) ➔ >:-3 =:-1 <:+9) 
          =:(1·6·8 ⟘ 3·4·11) ➔ >:-4 =:+12 <:+11) 
          <:(3 ⟘ 10) ➔ >:0 =:-2 <:+10)))


We can solve much larger puzzles. We can do 39 balls with 4 weighings:

In [19]:
do(Puzzle(39, 4))

(1·2·3·4·5·6·7·8·9·10·11·12·13 ⟘ 27·28·29·30·31·32·33·34·35·36·37·38·39) ➔ 
     >:(3·5·8·9·14·16·18·26·34·36·37·39 ⟘ 2·4·10·12·13·17·19·22·27·30·31·32) ➔ 
          >:(4·6·7·14·19·20·23·24·28·34·35·37 ⟘ 3·9·12·15·17·18·22·26·30·31·32·38) ➔ 
               >:(6·9·12·14·17·19·21·22·32·36 ⟘ 8·13·15·16·20·23·30·33·35·38) ➔ >:-30 =:-31 <:-32) 
               =:(7·8·11·16·17·19·21·24·27·34·36 ⟘ 2·6·12·14·15·18·26·30·33·37·39) ➔ >:+8 =:+5 <:-27) 
               <:(9·12·19·29·30 ⟘ 3·8·15·23·28) ➔ >:+9 =:0 <:+3)) 
          =:(8·9·13·20·26·27·29·30·34·35 ⟘ 1·5·10·18·19·25·28·32·33·38) ➔ 
               >:(3·7·14·16·20·25·27·29·30·33·34 ⟘ 2·4·5·8·12·13·24·31·35·38·39) ➔ >:-38 =:-28 <:-33) 
               =:(8·11·15·36·37·38 ⟘ 2·7·19·30·31·34) ➔ >:+11 =:+6 <:+7) 
               <:(1·2·3·9·10·17·26·28·34·35 ⟘ 6·7·12·15·16·22·24·30·32·37) ➔ >:+1 =:-29 <:-35)) 
          <:(1·2·3·8·13·21·25·26·29·33·37·38·39 ⟘ 6·7·12·16·17·19·23·24·27·28·30·31·34) ➔ 
               >:(3·6·7·16·17·18·20·22·24·25·27·

We can do 26 balls in 3 weighings if we know that no ball can be heavier:

In [20]:
do(Puzzle(26, 3, {-1}))

(1·2·3·4·5·6·7·8·9 ⟘ 18·19·20·21·22·23·24·25·26) ➔ 
     >:(3·7·8·9·14·16·20·21·23 ⟘ 2·4·6·10·12·13·18·19·24) ➔ 
          >:(3·8·16·19·21·22·25 ⟘ 4·5·9·10·11·20·24) ➔ >:-24 =:-18 <:-19) 
          =:(3·15·20·25 ⟘ 6·12·22·24) ➔ >:-22 =:-26 <:-25) 
          <:(1·3·7·9·21·22·24 ⟘ 2·8·13·14·17·18·23) ➔ >:-23 =:-20 <:-21)) 
     =:(7·11·15·17·21 ⟘ 2·8·10·14·20) ➔ 
          >:(4·9·16·18·20·22 ⟘ 2·5·6·8·14·26) ➔ >:-14 =:-10 <:0) 
          =:(13·15·19·20·21·22·23·25·26 ⟘ 1·4·7·9·10·12·14·18·24) ➔ >:-12 =:-16 <:-13) 
          <:(5·7·9·14·17·19·21 ⟘ 1·4·6·11·13·18·25) ➔ >:-11 =:-15 <:-17)) 
     <:(2·5·8·10·12·16·19·24·26 ⟘ 1·3·6·13·15·18·22·23·25) ➔ 
          >:(1·2·11·22 ⟘ 3·14·15·19) ➔ >:-3 =:-6 <:-1) 
          =:(6·9·12·13·17·21 ⟘ 1·2·4·5·14·16) ➔ >:-4 =:-7 <:-9) 
          <:(3·5·9·11·17·22·24·26 ⟘ 7·8·12·15·16·18·19·23) ➔ >:-8 =:-2 <:-5)))


# What's Next?

- What other puzzles can you solve?
- Can you make a table of solvable and unsolvable puzzles?
- What happens when it is a possibility that *no* ball is odd?
- What happens when it is a possibility that *two* balls are odd?
- Can you prove which puzzles are unsolvable? Can you modify the `find_tree` function so that it only fails when the puzzle really is unsolvable, and not when it happens to have bad luck with `random`?

It turns out that our solution for the 12-ball problem can handle the case where there is no oddball. It correctly returns 0, meaning there is no oddball, when given 0:

In [21]:
follow(solve(p12), oddball=0)

0

Three weighings can theoretically handle 27 possibilities. Can we solve the 13-ball problem, which has 26 possibilities? **No**, because there is no first weighing that partitions the 26 possibilities into 9/9/8; the best we can do is partition them into 8/10/8 or 10/6/10:

In [22]:
p13 = Puzzle(13)
partition([1, 2, 3, 4], [10, 11, 12, 13], p13.oddballs)

{'gt': [1, 2, 3, 4, -10, -11, -12, -13],
 'eq': [5, -5, 6, -6, 7, -7, 8, -8, 9, -9],
 'lt': [-1, -2, -3, -4, 10, 11, 12, 13]}

In [23]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], p13.oddballs)

{'gt': [1, 2, 3, 4, 5, -9, -10, -11, -12, -13],
 'eq': [6, -6, 7, -7, 8, -8],
 'lt': [-1, -2, -3, -4, -5, 9, 10, 11, 12, 13]}

What else can you discover?